# LSH Algorithm Improvement By Applying Bitmap Indexing

In [51]:
import argparse
import sys
from os import listdir
from os.path import isfile, join
from typing import Dict, List, Optional, Tuple
import imagehash
from PIL import Image
import os, os.path
import cv2
from collections import Counter
import scipy as sp
import numpy as np # Import numpy library 
from skimage.feature import hog # Import Hog model to extract features
from sklearn.metrics import confusion_matrix # Import confusion matrix to evaluate the performance
import pandas as pd
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark.conf import SparkConf
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split
from pyspark.sql.functions import lit
from pyspark.sql.types import *
from pyspark.sql.functions import sum as _sum

In [34]:
imgs = []
y = []
file_size = []
k = 0
path = "./data/101_ObjectCategories" # Give the dataset path here

##  Data Preprocessing:
1. Load the images using cv2
2. Image resize
3. Feature extraction: BGR to Gray conversion 
4. Feature extraction: Histogram of Oriented Gradients(HOG)

In [35]:
folder = os.listdir(path) # from the given path get the file names such as accordion, airplanes etc..
for file in folder: # for every file name in the given path go inseide that directory and get the images
    subpath = os.path.join(path,file)  # Join the name of these files to the previous path 
    
    files = os.listdir(subpath) # Take these image names to a list called files
    j = 0
    for i in range(np.size(files)): # now we shall loop through these number of files
        
        im = cv2.imread(subpath+'/'+files[0+j]) # Read the images from this subpath
        
        imgs.append(im) # append all the read images to a list called imgs
        y.append(k) # generate a labe to every file and append it to labels list

        j += 1
        if (j == (np.size(files))):
            file_size.append(j)
   
    k += 1
     
y = np.array(y).tolist()
ix = []
for index, item in enumerate(imgs):
    if (np.size(item) == 1):
        ix.append(index)
        del imgs[index]
        
for index, item in enumerate(y):
    for v in range(np.size(ix)):
        if (index == ix[v]):
            del y[index]
        
y = np.array(y).astype(np.float64) 

# Function to convert an image from color to grayscale
def resize_(image):
    u = cv2.resize(image,(256,256))
    return u

def rgb2gray(rgb):
    gray = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)
    return gray

def fd_hog(image):
    fd = hog(image, orientations=8, pixels_per_cell=(64, 64),
                        cells_per_block=(2, 2))
    
    return fd

In [36]:
a=[]
import progressbar
with progressbar.ProgressBar(max_value=len(imgs)) as bar:
    i=1
    for img in imgs:
        b=resize_(img)
        c=rgb2gray(b)   
        d=fd_hog(c)
        a.append(d)
        bar.update(i)
        i+=1
df = pd.DataFrame(a)
df['lable'] = y
id_ = np.arange(1,len(df)+1,1)
df['id'] = id_
X = df.values

spark = SparkSession.builder \
 .master("local") \
 .appName("Image Retrieval") \
 .config("spark.some.config.option", "some-value") \
 .getOrCreate()


100% (9176 of 9176) |####################| Elapsed Time: 0:01:55 Time:  0:01:55


In [37]:
print("HOG diamension: ")
len(a[0])

HOG diamension: 


288

In [66]:
def getBestPerformance(numOfTest, bucketLength,numHashTables,numOfNeighbor):
 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    Train = map(lambda x: (int(x[-1]),int(x[-2]),Vectors.dense(x[:-2])), X_train)
    Train_df = spark.createDataFrame(Train,schema=['id','label',"features"])
    Test = map(lambda x: (int(x[-1]),int(x[-2]),Vectors.dense(x[:-2])), X_test)
    Test_df = spark.createDataFrame(Test,schema=['id','label',"features"])

    brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", 
                                      bucketLength=bucketLength,numHashTables=numHashTables)
    
    
    model = brp.fit(Train_df)
    model.transform(Train_df)
  
    nnToMajorityAccMap = {}
    nnToWeightedAccMap = {}
    with progressbar.ProgressBar(max_value = numOfTest) as bar:
        for i in range(0, numOfTest):
            Catg = X_test[i][-2]
            key = Vectors.dense(X_test[i][0:-2])
            # Choose the Last one of numOfNeighbor, the biggest one 
            result = model.approxNearestNeighbors(Train_df, key, numOfNeighbor[-1], distCol="EuclideanDistance")
            # Conver pySpark framework colunm to python list
            labelList = result.select("label").rdd.flatMap(lambda x: x).collect()
            # Build a distance dataframe for the distance between query and result
            result1 = result.select('label', 'EuclideanDistance').collect()
            df_weighted = pd.DataFrame()
            df_weighted['Label'] = [int(row['label']) for row in result1]
            df_weighted['EuclideanDistance'] = [float(row['EuclideanDistance']) for row in result1]
            df_weighted['Weight'] = 1 / df_weighted['EuclideanDistance']
            # slice LableList into differnt length subLists 
            nnList = []
            for numberNN in numOfNeighbor:
                slicedList = labelList[0:numberNN]
                nnList.append(slicedList)
                
            for index in range(0, len(nnList)):
                majority_vote = Counter(nnList[index]).most_common(1)[0][0]
                if  Catg == majority_vote:
                    key = numOfNeighbor[index]
                    if key in nnToMajorityAccMap:
                        nnToMajorityAccMap[key] = nnToMajorityAccMap.get(key) + 1
                    else:
                        nnToMajorityAccMap[key] = 1
                        
            for n in numOfNeighbor:
                weighted_result = df_weighted.head(n).groupby('Label')['Weight'].sum()
                df_weighted_result = pd.DataFrame(weighted_result)
                df_weighted_result = df_weighted_result.reset_index()
                df_weighted_result.sort_values(by='Weight',inplace=True,ascending=False)
                weighted_vote = df_weighted_result.iloc[0]['Label'].astype('int')
                if  Catg == weighted_vote:
                    key = n
                    if key in nnToWeightedAccMap:
                        nnToWeightedAccMap[key] = nnToWeightedAccMap.get(key) + 1
                    else:
                        nnToWeightedAccMap[key] = 1
                        
            bar.update(i)
        # calucate accuracy base on Majority Vote
        for key in nnToMajorityAccMap:
            nnToMajorityAccMap[key] = nnToMajorityAccMap.get(key) / numOfTest
        # calucate accuracy base on Weighted Vote
        for key in nnToWeightedAccMap:
            nnToWeightedAccMap[key] = nnToWeightedAccMap.get(key) / numOfTest    
    return nnToMajorityAccMap, nnToWeightedAccMap

In [67]:
#set Param 
bucketLengthList = np.arange(0, 50, 10)
numHashTablesList = np.arange(0, 150, 30)
bucketLengthList[0] = 1
numHashTablesList[0] = 1
#make sure the last element of numOfNeighborList is the largest
numOfNeighborList = [1, 3, 5, 7, 15, 21, 25];
numOfTest = 100
print("Checking bucketLength Param:")
print(bucketLengthList)
print("Checking numHashTablesList Param:")
print(numHashTablesList)

Checking bucketLength Param:
[ 1 10 20 30 40]
Checking numHashTablesList Param:
[  1  30  60  90 120]


In [ ]:
bucketLengthList_para=[]
numHashTablesList_para=[]
resultMList = []
resultWList = []
for i in bucketLengthList:
    for j in numHashTablesList:
            resultM, resultW = getBestPerformance(numOfTest ,i, j, numOfNeighborList)
            print( "bucketLen:" + str(i) + "  #Hashtable:" + str(j) +  "  Majority Vote Acc: " + str(resultM))
            print( "bucketLen:" + str(i) + "  #Hashtable:" + str(j) +  "  Weighted Vote Acc: " + str(resultW))
            bucketLengthList_para.append(i)
            numHashTablesList_para.append(j)
            resultMList.append(resultM)
            resultWList.append(resultW)

100% (100 of 100) |######################| Elapsed Time: 0:02:51 Time:  0:02:51


bucketLen:1  #Hashtable:1  Majority Vote Acc: {1: 0.43, 3: 0.44, 5: 0.43, 7: 0.45, 15: 0.45, 21: 0.42, 25: 0.43}
bucketLen:1  #Hashtable:1  Weighted Vote Acc: {1: 0.43, 3: 0.44, 5: 0.44, 7: 0.46, 15: 0.47, 21: 0.42, 25: 0.45}


N/A% (0 of 100) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

In [ ]:
df_result = pd.DataFrame()
df_result['BucketLength'] = bucketLengthList_para
df_result['NumHashTables'] = numHashTablesList_para
# conver List of map to panda dataframework
df_result_map = pd.DataFrame(resultMList)
df_result_map2 = pd.DataFrame(resultWList)
df_result = pd.concat([df_result, df_result_map], axis=1, join='inner')
df_result = pd.concat([df_result, df_result_map2], axis=1, join='inner')

# df_result["Acc"] = resultList
# df_result = df_result.sort_values(by=['Acc'],ascending=False)
df_result.to_csv('./result2.csv') #Chang the name every you wanna sava a file

In [ ]:
df_result

# !!!Skip all the code below !!!

## Split data
Split the data to training and validation data. We choose 70% for training and 30% for validation purposes.

In [6]:
%%time
# append 'label' and 'id' to the last two colunms
df = pd.DataFrame(a)
df['lable'] = y
id_ = np.arange(1,len(df)+1,1)
df['id'] = id_
X = df.values

CPU times: user 1.74 s, sys: 428 ms, total: 2.16 s
Wall time: 2.97 s


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Using PySpark to retrieve similar images

In [ ]:
spark = SparkSession.builder \
     .master("local") \
     .appName("Image Retrieval") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()

In [8]:
Train = map(lambda x: (int(x[-1]),int(x[-2]),Vectors.dense(x[:-2])), X_train)
Train_df = spark.createDataFrame(Train,schema=['id','label',"features"])

In [9]:
Test = map(lambda x: (int(x[-1]),int(x[-2]),Vectors.dense(x[:-2])), X_test)
Test_df = spark.createDataFrame(Test,schema=['id','label',"features"])

In [10]:
Train_df.show(n = 2)

+----+-----+--------------------+
|  id|label|            features|
+----+-----+--------------------+
|5960|   80|[0.00363684489770...|
|1536|   18|[0.06071563370892...|
+----+-----+--------------------+
only showing top 2 rows



In [11]:
%reload_ext memory_profiler
def toMeasureMemo():
    brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes",bucketLength=2,numHashTables=3)
    model = brp.fit(Train_df)
    print("The hashed dataset where hashed values are stored in the column 'hashes':")
    model.transform(Train_df).show()
%memit  toMeasureMemo()

The hashed dataset where hashed values are stored in the column 'hashes':
+----+-----+--------------------+--------------------+
|  id|label|            features|              hashes|
+----+-----+--------------------+--------------------+
|5960|   80|[0.00363684489770...|[[0.0], [0.0], [-...|
|1536|   18|[0.06071563370892...|[[0.0], [0.0], [-...|
|8387|   95|[0.01072387506065...|[[-1.0], [0.0], [...|
|6169|   83|[0.00663216869878...|[[0.0], [0.0], [0...|
|4067|   55|[0.01922287915132...|[[-1.0], [-1.0], ...|
|2135|   23|[0.02042261440886...|[[0.0], [0.0], [0...|
|8246|   95|[0.02765804852049...|[[-1.0], [-1.0], ...|
|8690|   96|[0.03719083421779...|[[-1.0], [0.0], [...|
|6724|   88|[0.06854134811831...|[[-1.0], [0.0], [...|
|  26|    0|[0.04798466879403...|[[-1.0], [0.0], [...|
|6432|   88|[0.01443607483278...|[[0.0], [0.0], [-...|
|2392|   28|[0.02645665091604...|[[-1.0], [0.0], [...|
|1580|   18|[0.16710970919873...|[[-1.0], [0.0], [...|
|1388|   18|[0.04740743977419...|[[0.0], [0.0]

In [ ]:
key = Vectors.dense(X_test[0][0:-2])

In [ ]:
key

In [ ]:
X_test[0][-2]

In [ ]:
print("Approximately searching Train_df for 2 nearest neighbors of the key:")
model.approxNearestNeighbors(Train_df, key, 5).show()

In [ ]:
# result_id = result.select('label',).collect()
# result_id[0].label

In [ ]:
# print("Approximately joining Train_df and Test_df on Euclidean distance smaller than 1:")
# model.approxSimilarityJoin(Train_df, Test_df, 1.1, distCol="EuclideanDistance")\
#     .select(col("datasetA.id").alias("Train_df"),
#             col("datasetB.id").alias("Test_df"),
#             col("EuclideanDistance")).show(30)

In [ ]:
accuracy = 0
numOfNeighbor = 5
numOfTest= 5
accList = []
with progressbar.ProgressBar(max_value=numOfTest) as bar:
    for i in range(0, numOfTest):
        Catg = X_test[i][-2]
        key = Vectors.dense(X_test[i][0:-2])
        result = model.approxNearestNeighbors(Train_df, key, numOfNeighbor)
        temp = Counter([int(row['label']) for row in result.collect()])
        if  Catg in temp:
            accuracy += temp.get(Catg)/ numOfNeighbor
            accList.append(temp.get(Catg)/ numOfNeighbor)
        else:
            accList.append(0)
        bar.update(i)
    accuracy /= numOfTest

In [ ]:
accuracy

In [ ]:
print(accList)

In [ ]:
# from matplotlib.pyplot import imshow
# imshow(imgs[4795])

In [32]:
%load_ext memory_profiler
def my_func():
    a = [1] * (10 **6)
    b = [2] * (2 * 10 ** 7)
    del b
    return a
%memit my_func()

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
peak memory: 288.71 MiB, increment: 160.33 MiB


[[1], [1, 2, 3], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5, 6, 7]]